In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

URL = "https://en.wikipedia.org/wiki/2023%E2%80%9324_UEFA_Champions_League"

def scrape_ucl_results(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    tables = soup.find_all("table", {"class": "fevent"})

    matches = []

    for table in tables:
        rows = table.find_all("tr")

        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 5:
                continue

            date = cols[0].get_text(" ", strip=True)
            home = cols[1].get_text(" ", strip=True)
            score = cols[2].get_text(" ", strip=True)
            away = cols[3].get_text(" ", strip=True)

            # Extract goals from score
            score_match = re.search(r"(\d+)\s*–\s*(\d+)", score)
            if not score_match:
                continue

            home_goals = int(score_match.group(1))
            away_goals = int(score_match.group(2))

            matches.append({
                "date": date,
                "home_team": home,
                "away_team": away,
                "home_goals": home_goals,
                "away_goals": away_goals
            })

    df = pd.DataFrame(matches)
    return df


df = scrape_ucl_results(URL)
print(df.head())

# df.to_csv("ucl_2023_24_results.csv", index=False)
print("Saved as ucl_2023_24_results.csv")


Retry 1/5 failed: HTTPSConnectionPool(host='www.uefa.com', port=443): Read timed out. (read timeout=15)
Retry 2/5 failed: HTTPSConnectionPool(host='www.uefa.com', port=443): Read timed out. (read timeout=15)
Retry 3/5 failed: HTTPSConnectionPool(host='www.uefa.com', port=443): Max retries exceeded with url: /uefachampionsleague/fixtures-results/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002BD525FD410>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Retry 4/5 failed: HTTPSConnectionPool(host='www.uefa.com', port=443): Read timed out. (read timeout=15)
Retry 5/5 failed: HTTPSConnectionPool(host='www.uefa.com', port=443): Read timed out. (read timeout=15)


Exception: UEFA blocked the request after several retries.